# Homework Week 05

## Thomas Plaisier

For homework this week, your job is to do the following:

## Vary the `parallel_threads` and `prefetch_buffer_size` parameters at the top of [train_resnet34.py](train_resnet34.py) and make a table of how the mean image rate changes.

I tested various combinations of thread count and buffer count to see which one performed best. Out of interest, I tested the combinations on both the single GPU and full node setups. I expected that for lower combinations of thread/buffer counts the single GPU setup would perform better because of lower overhead, but that for higher combinations the full node would be faster because of extra available hardware.
The results are shown at the bottom of the notebook. As expected, increasing either thread count or buffer count generally improved processing speed. However, increasing thread count quickly reached a point of diminishing or even negative returns: for example, going from T1/B16 to T4/B16 increased processing speed almost 5-fold but further increasing the thread count gave only marginal improvements. Increasing thread count beyond 32 even decreased average processing speed. For the full node setup this effect was even more pronounced. Increasingly larger buffer counts did not hit the point of negative returns for the tested combinations, with a few exceptions such as single GPU T128/B16 to T128/B32. 
Counter to my expectation, the full node setup scored worse than the single GPU setup on every tested combination of thread count and buffer count. It seems that the overhead of having to coordinate the training loops over the extra hardware actually slows down the process, rather than making effective use of the extra cores. This is most likely due to a lack of optimization of the code for this particular setup and the simplicity of my testing scheme, and not an inherent problem with the full node setup. 

In [1]:
import numpy as np
import pandas as pd
import re
import os
import glob

In [2]:
# Load log file.
single_log = "10111422.output"

file_read = open(single_log, "r")

lines = file_read.readlines()

profile_list = []
result_list = []
profile_idx = 0
result_idx = 0

profile_expression = 'Profiling to logdir/m(?P<m>\d+)_w(?P<w>\d+)_p(?P<p>\d+)_.*'
profile_object = re.compile(profile_expression)

result_expression = 'mean image_s\s+\=\s+(?P<mean>\w+\.\w+),\s+standard\s+deviation:\s+(?P<std>\w+\.\w+)'
result_object = re.compile(result_expression)

for line in lines:
    is_profile = profile_object.search(line)
    if is_profile is not None:
        m = is_profile.group('m')
        w = is_profile.group('w')
        p = is_profile.group('p')
        profile_list.append([int(m), int(w), int(p)])

    is_result = result_object.search(line)
    if is_result is not None:
        mean = is_result.group('mean')
        result_list.append(float(mean))

file_read.close()

single_table = pd.DataFrame(dtype = float)
this_index = 0
for this_set in profile_list:
    single_table.at[f'T{this_set[0]}', f'B{this_set[2]}'] = result_list[this_index]
    this_index += 1

In [4]:
# Load log file.
full_log = "10111420.output"

file_read = open(full_log, "r")

lines = file_read.readlines()

profile_list = []
result_list = []
profile_idx = 0
result_idx = 0

profile_expression = 'Profiling to logdir/m(?P<m>\d+)_w(?P<w>\d+)_p(?P<p>\d+)_.*'
profile_object = re.compile(profile_expression)

result_expression = 'mean image_s\s+\=\s+(?P<mean>\w+\.\w+),\s+standard\s+deviation:\s+(?P<std>\w+\.\w+)'
result_object = re.compile(result_expression)

for line in lines:
    is_profile = profile_object.search(line)
    if is_profile is not None:
        m = is_profile.group('m')
        w = is_profile.group('w')
        p = is_profile.group('p')
        profile_list.append([int(m), int(w), int(p)])

    is_result = result_object.search(line)
    if is_result is not None:
        mean = is_result.group('mean')
        result_list.append(float(mean))

file_read.close()

full_table = pd.DataFrame(dtype = float)
this_index = 0
for this_set in profile_list:
    full_table.at[f'T{this_set[0]}', f'B{this_set[2]}'] = result_list[this_index]
    this_index += 1

In [5]:
print('Values in img/s.')
single_table

Values in img/s.


,B1,B4,B8,B16,B32
T1,110.29,169.66,183.95,227.46,372.37
T4,413.46,566.72,697.53,1016.73,1071.52
T8,453.92,572.20,722.75,1054.92,1244.71
T16,564.97,669.90,820.23,1101.25,1276.44
T32,718.74,812.96,920.00,1159.21,1197.00
T128,857.34,935.99,954.46,1008.53,967.24
T256,778.71,781.92,770.99,772.92,772.67


In [6]:
print('Values in img/s.')
full_table

Values in img/s.


,B1,B4,B8,B16,B32
T1,105.31,144.78,146.32,184.42,322.24
T4,402.23,424.21,600.39,866.02,866.94
T8,400.42,515.14,620.30,903.99,952.42
T16,526.35,617.41,771.07,1004.55,889.49
T32,680.93,771.03,849.88,949.05,579.33
T128,841.81,861.41,776.32,645.79,655.22
T256,720.05,721.73,623.08,540.01,705.91
